In [24]:
import pymssql as pymssql
import pandas as pd
import glob
import numpy as np
import geopandas as gp
import shapely.geometry as geom

#Connection à la base 
cnxn=pymssql.connect(server='info-mssql-etd', user='etd14',password='x578tcb5',database='BDEquipe14')

# # On déclare un curseur
cursor=cnxn.cursor()

# #csv contenant le nom de tous les pays avec son code et la région auquel il appartient 
countryRegion= pd.read_csv('pays-region.csv',delimiter=';')

# #csv contenant la lat , lon et le code pays 
latLonCountry= pd.read_csv('LatLonPays.csv')

# #csv contenant  les GDP
GDP=pd.read_csv('GDP2.csv')
GDP = GDP[['Country Code'] + [str(i) for i in range(1960, 2023)]].fillna(0)

#csv contenant  les GHG
GHG=pd.read_csv('GHG.csv')
GHG = GHG[['Country Code'] + [str(i) for i in range(1960, 2023)]].fillna(0)
#csv contenant  les population
POP=pd.read_csv('population.csv')
POP = POP[['Country Code'] + [str(i) for i in range(1960, 2023)]].fillna(0)

sources= pd.read_csv('ENS&SCT/sources.csv',delimiter=';')
sectors=pd.read_csv('ENS&SCT/secteurs.csv')

df = gp.read_file('countries.geojson')

def linkCoordToCountry(name):
    df2 = pd.read_csv(name)
    df2['coords'] = list(zip(df2['lon'],df2['lat']))
    df2['coords'] = df2['coords'].apply(geom.Point)
    points = gp.GeoDataFrame(df2, geometry='coords', crs=df.crs)
    pointInPolys = gp.tools.sjoin(points, df, op="within", how='left')
    return pointInPolys[["lat","lon","crs","total","ISO_A3"]].fillna("")

seaLvl=linkCoordToCountry("sealvl.csv")
TX35=linkCoordToCountry("TX35.csv")
meanTemp=linkCoordToCountry("meanTemp.csv")
meanTempChange=linkCoordToCountry("meanTempChange.csv")
precipit=linkCoordToCountry("totalPrecipit.csv")

def delete(table):
    cursor.execute("DELETE FROM "+table)
    cnxn.commit()

# Fonction pour importer les pays
def importCountries():
    pays = countryRegion[['name', 'geo']]
    for index, row in pays.iterrows():
        cursor.execute("INSERT INTO  T_COUNTRY_CTR (CTR_ID, CTR_NAME) VALUES (%s, %s)", (row['geo'][:3].upper(), row['name'][:32]))
    cnxn.commit()

# Fonction pour importer les régions
def importRegions():
    region = countryRegion[['eight_regions']]
    region = region.drop_duplicates()
    region = region.dropna()
    for index, row in region.iterrows():
        cursor.execute("INSERT INTO T_REGION_REG (REG_NAME) VALUES (%s)", (row['eight_regions'][:255]))
    cnxn.commit()

# Fonction pour importer les pays dans les régions
def importIsOn():
    isOn = countryRegion[['eight_regions','geo']]
    isOn = isOn.dropna()
    for index, row in isOn.iterrows():
        cursor.execute("INSERT INTO  IS_ON (REG_ID,CTR_ID) VALUES ((select REG_ID from T_REGION_REG where REG_NAME=%s),%s)", (row['eight_regions'][:255],row['geo'][:3].upper()))
    cnxn.commit()

# Fonction pour importer les indicateurs
def importIndicator():
    for index, row in POP.iterrows():
        ctr_id = row['Country Code'][:3]
        for year in range(1960, 2023):
            year_str = str(year)
            gdp_value = GDP.loc[GDP['Country Code'] == row['Country Code'], year_str].values[0]
            ghg_value = GHG.loc[GHG['Country Code'] == row['Country Code'], year_str].values[0]
            pop_value = row[year_str]
            
            cursor.execute("SELECT COUNT(*) FROM T_COUNTRY_CTR WHERE CTR_ID = %s", (ctr_id,))
            if cursor.fetchone()[0] > 0:
                cursor.execute("INSERT INTO T_INDICATOR_IND (CTR_ID, IND_GDP, IND_GHG, IND_POPULATION, IND_YEAR) VALUES (%s, %s, %s, %s, %s)", 
                               (ctr_id, gdp_value, ghg_value, pop_value, year))
            else:
                print(f"Le CTR_ID {ctr_id} n'existe pas dans T_COUNTRY_CTR")
    cnxn.commit()

# Fonction pour importer les données géographiques
def importGeo():
        for index, row in seaLvl.iterrows():
            cursor.execute("SELECT COUNT(*) FROM T_COUNTRY_CTR WHERE CTR_ID = %s", (row['ISO_A3'],))
            if cursor.fetchone()[0] > 0:
                cursor.execute("INSERT INTO T_GEOGRAPHICALDATA_GEO (CTR_ID, GEO_CRS, GEO_MEANTEMP, GEO_TX35, GEO_SEALVL, GEO_MEANTEMPCHANGE, GEO_TOTALPRECIPIT, GEO_LAT, GEO_LON) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                            (row['ISO_A3'], row['crs'], meanTemp.loc[index, 'total'], TX35.loc[index, 'total'], seaLvl.loc[index, 'total'], meanTempChange.loc[index, 'total'], precipit.loc[index, 'total'], row['lat'], row['lon']))
            else:
                print(f"Le CTR_ID {row['ISO_A3']} n'existe pas dans T_COUNTRY_CTR")
        cnxn.commit()

# Fonction pour importer les sources d'énergie
def importSources():
    for index, row in sources.iterrows():
        cursor.execute("INSERT INTO  T_ENERGYSOURCE_ENS (ENS_NAME,ENS_ISFOSSIL) VALUES (%s,%s)", (row['name'][:32], row['isFossil']))
    cnxn.commit()

# Fonction pour importer les secteurs
def importSectors():
    for index, row in sectors.iterrows():
        cursor.execute("INSERT INTO  T_SECTOR_SCT (SCT_NAME) VALUES (%s)", (row['name'][:32]))
    cnxn.commit() 

# Fonction pour obtenir le multiplicateur de CO2
def get_co2_multiplier(country_code):
    match country_code:
        case 'FRA':
            return 0.31
        case 'DNK':
            return 0.5
        case 'DEU':
            return 0.441
        case 'CIV':
            return 0.62
        case 'CHN':
            return 0.53
        case 'IND':
            return 0.82
        case 'USA':
            return 0.53

# Fonction pour importer les émissions de CO2 par secteur
def importSectorPays():
    filenames = glob.glob("ENS&SCT/EmissionSecteur/*.csv")
    dfs = []
    for filename in filenames:
        country_code = filename[-7:-4].upper()  # Extrayant les 3 dernières lettres en majuscules
        df = pd.read_csv(filename,delimiter=';',index_col=0)
        df = df.rename_axis("date").reset_index()
        df["date"] = df['date'].str[0:4]
        df['country_code'] = country_code
        df['co2_muli'] = get_co2_multiplier(country_code)
        df['co2_muli'] = df['co2_muli'].astype(float)
        dfs.append(df)
        
    merged_df = pd.concat(dfs, ignore_index=True)
    for index, row in merged_df.iterrows():
            date = row['date']
            for sector, emission in row.items():
                if sector not in ['date', 'country_code', 'co2_muli']:
                    cursor.execute("SELECT SCT_ID FROM T_SECTOR_SCT WHERE SCT_NAME = %s", (sector,))
                    sector_id = cursor.fetchone()[0]
                    twhtokwgh = float(emission.replace(',','.')) * 1000000000
                    co2emission = twhtokwgh * row['co2_muli']    
                    cursor.execute("INSERT INTO HAVE (CTR_ID, SCT_ID, CO2EMISSION, SECTORYEAR) VALUES (%s, %s, %s, %s)",
                                    (row['country_code'], sector_id, co2emission, date))
            cnxn.commit()

# Fonction pour importer les sources d'énergie par pays (Elle réalise la jointure sur de plusieurs fichiers afin d'obtenir les données d'émission et de sources d'énergie par pays)
def importSourcesPays():
    filenames_fe = glob.glob("ENS&SCT/EmissionSources/FE/*.csv")
    filenames_pec = glob.glob("ENS&SCT/EmissionSources/PEC/*.csv")
    filenames_pep = glob.glob("ENS&SCT/EmissionSources/PEP/*.csv")
    
    data_fe = {}
    data_pec = {}
    data_pep = {}
    
    for filename in filenames_fe:
        country_code = filename[-7:-4].upper()
        df = pd.read_csv(filename, delimiter=';', index_col=0)
        df = df.rename_axis("date").reset_index()
        df["date"] = df['date'].str[0:4]
        df['country_code'] = country_code
        df['co2_muli'] = get_co2_multiplier(country_code)
        df['co2_muli'] = df['co2_muli'].astype(float)
        data_fe[country_code] = df
    
    for filename in filenames_pec:
        country_code = filename[-7:-4].upper()
        df = pd.read_csv(filename, delimiter=';', index_col=0)
        df = df.rename_axis("date").reset_index()
        df["date"] = df['date'].str[0:4]
        df['country_code'] = country_code
        data_pec[country_code] = df

    for filename in filenames_pep:
        country_code = filename[-7:-4].upper()
        df = pd.read_csv(filename, delimiter=';', index_col=0)
        df = df.rename_axis("date").reset_index()
        df["date"] = df['date'].str[0:4]
        df['country_code'] = country_code
        data_pep[country_code] = df
    
    # Liaison des données d'émission de sources d'énergie avec les données de consommation et de production par pays
    for country_code in data_fe:
        df_fe = data_fe[country_code]
        df_pec = data_pec.get(country_code)
        df_pep = data_pep.get(country_code)
        
        for index, row in df_fe.iterrows():
            date = row['date']
            for source, value in row.items():
                if source not in ['date', 'country_code', 'co2_muli'] and source is not np.nan and not pd.isna(value):
                    cursor.execute("SELECT ENS_ID FROM T_ENERGYSOURCE_ENS WHERE ENS_NAME = %s", (source,))
                    result = cursor.fetchone()
                    if result:
                        source_id = result[0]
                        if isinstance(value, str):
                            value = value.replace(',', '.')
                        try:
                            twhtokwgh = float(value) * 1000000000
                        except ValueError:
                            continue
                        co2emission = twhtokwgh * row['co2_muli']
                        
                        consumption = 0
                        production = 0
                        
                        if df_pec is not None and source in df_pec.columns:
                            consumption_values = df_pec.loc[df_pec['date'] == date, source].values
                            if len(consumption_values) > 0 and not pd.isna(consumption_values[0]):
                                consumption = float(consumption_values[0].replace(',', '.') if isinstance(consumption_values[0], str) else consumption_values[0])
                        
                        if df_pep is not None and source in df_pep.columns:
                            production_values = df_pep.loc[df_pep['date'] == date, source].values
                            if len(production_values) > 0 and not pd.isna(production_values[0]):
                                production = float(production_values[0].replace(',', '.') if isinstance(production_values[0], str) else production_values[0])
                        
                        cursor.execute(
                            "INSERT INTO CAN_USE (CTR_ID, ENS_ID, Co2Emission, ENSCONSUMPTION, ENSPRODUCTION, SOURCEYEAR) VALUES (%s, %s, %s, %s, %s, %s)",
                            (row['country_code'], source_id, co2emission, consumption, production, date)
                        )
    cnxn.commit()

def clearAll():
    delete("CAN_USE")
    delete("HAVE")
    delete("T_GEOGRAPHICALDATA_GEO")
    delete("T_ENERGYSOURCE_ENS")
    delete("T_SECTOR_SCT")
    delete("T_INDICATOR_IND")
    delete("IS_ON")
    delete("T_REGION_REG")
    delete("T_COUNTRY_CTR")

def importAll():
    clearAll()
    importCountries()
    importRegions()
    importIsOn()
    importIndicator()
    importGeo()
    importSources()
    importSectors()
    importSectorPays()
    importSourcesPays()

/mnt/netta/users/ojfrancois/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/mnt/netta/users/ojfrancois/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/mnt/netta/users/ojfrancois/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/mnt/netta/users/ojfrancois/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` par

,ID_CAN_USE,CTR_ID,ENS_ID,CO2EMISSION,ENSCONSUMPTION,ENSPRODUCTION,SOURCEYEAR
0,31,FRA,1,3.113606e+10,0.000000,0.000000,1971
1,32,FRA,2,5.750623e+10,0.000000,282.209381,1971
2,33,FRA,3,0.000000e+00,0.000000,0.000000,1971
3,34,FRA,4,3.913502e+10,0.000000,0.000000,1971
4,35,FRA,5,2.514371e+10,0.000000,76.500679,1971
...,...,...,...,...,...,...,...
531,562,USA,4,2.003841e+12,0.000000,0.000000,2015
532,563,USA,5,2.054157e+12,8269.321289,8232.885742,2015
533,564,USA,6,1.577342e+10,45.478786,45.478786,2015
534,565,USA,7,3.364257e+10,0.000000,0.000000,2015
